In [1]:
import numpy as np
import pandas as pd
import sublevel as sb

Let's take a test function -- "camel function with three minima":

In [2]:
def Func(x, y):
    return 2*x**2 - 1.05*x**4+x**6/6+x*y + y**2

## Random point cloud

We generate points and calculate function values at these points. It's important that generated numpy array would contain real numbers with the same accuracy like float32, otherwise the function's behaviour would be unpredictable!

In [3]:
# 500000 random points in the square [-2, 2]*[-2, 2]
coordinates = (4 * np.random.rand(500000,2) - 2).astype('float32')
values = Func(coordinates[:,0], coordinates[:,1])

In [4]:
res = sb.random(values, coordinates)

It's convenient to represent the result as pandas.DataFrame

In [5]:
Res = pd.DataFrame.from_dict(res)

In [6]:
Res.loc[Res['Number of point in dead cluster'] > 50]

,birth value,death value,birth of swallowed cluster,Id of dead minimum,Id of saddle,Id of swallowed minimum,Number of point in dead cluster,Number of point in swallowed cluster
610,2.146926,2.266414,0.000059,21670,326949,112286,51,324118
2021,0.298662,0.877353,0.000059,69758,20285,112286,29106,80677
3341,0.000059,inf,inf,112286,-1,-1,499616,-1
4842,0.902148,0.974002,0.000059,161944,344323,112286,54,164228
7276,0.577370,0.630223,0.298662,241196,22346,69758,53,14678
7957,3.239271,3.328517,0.000059,263575,118187,112286,51,394266
8780,2.608188,2.758301,0.000059,291632,26000,112286,65,360487
12534,0.298641,0.877432,0.000059,416439,333834,112286,29304,109823


We can see 3 minima and several noisy clusters.

## Rectangular grid for the points

Here we have 2 functions: sb.grid и sb.grid_shape. The first one takes np.array tensor that consists of function values on the grid, second one is one-dimensional np.array that contains function values on the grid sorted by points lexicographic order, and np.array that contains dimensions of axis. Let's demonstrate this via examples:

In [25]:
def ndim_grid(start,stop, steps):
    ndims = len(start)
    L = [np.linspace(start[i],stop[i], steps[i], dtype = np.float32) for i in range(ndims)]
    return np.hstack((np.meshgrid(*L))).swapaxes(0,1).reshape(ndims,-1).T

The function ndim_grid generates the list of coordinates in lexicographic order on the grid between points "start" and "stop" with the number of separations "steps". Let's apply it to our function:

In [32]:
min = np.array([-2.0, -2.0], dtype = np.float32)
max = np.array([2.0, 2.0], dtype = np.float32)
shape = np.array([3000, 2000], dtype = np.int32)
grid = ndim_grid(min, max, shape)
values = Func(grid[:,0], grid[:,1])

In [33]:
ans = sb.grid_shape(values, shape)

In [34]:
pd.DataFrame.from_dict(ans)

,birth value,death value,birth of swallowed cluster,Id of dead minimum,Id of saddle,Id of swallowed minimum,Number of point in dead cluster,Number of point in swallowed cluster
0,0.298640,0.877361,0.000001,379436,1395267,2999000,347355,972343
1,0.000001,inf,inf,2999000,-1,-1,6000000,-1
2,0.000001,0.000003,0.000001,3000999,2998999,2999000,1,2
3,0.298640,0.877361,0.000001,5620563,4604732,2999000,347355,1319703


We could also get the same result using function sb.grid, giving to it the only np.array of values and corresponding dimensions:

In [43]:
val = values.reshape(3000, 2000)

In [44]:
ans = sb.grid(val)
pd.DataFrame.from_dict(ans)

,birth value,death value,birth of swallowed cluster,Id of dead minimum,Id of saddle,Id of swallowed minimum,Number of point in dead cluster,Number of point in swallowed cluster
0,0.298640,0.877361,0.000001,379436,1395267,2999000,347355,972343
1,0.000001,inf,inf,2999000,-1,-1,6000000,-1
2,0.000001,0.000003,0.000001,3000999,2998999,2999000,1,2
3,0.298640,0.877361,0.000001,5620563,4604732,2999000,347355,1319703


This data contains one cluster of noise.